In [1]:
import sys
import json
import requests
import time
import argparse
import pandas as pd
import numpy as np
import pymongo

import random

import boto3
import boto.s3.connection
from boto.s3.key import Key
from src.Airbnb_Scraping_tools import *
from urllib.parse import urlparse
from urllib.request import urlretrieve 

In [2]:
%load_ext autoreload
%autoreload 2

# Load in data

In [3]:
s3_path = 'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [4]:
s3_path

'https://s3-us-west-2.amazonaws.com/chen-gal-test/AirbnbData/'

In [6]:
folder = 'Chicago-Illinois-US'
file_tmp = pd.read_csv(s3_path+folder+'/listings.csv.gz',compression = 'gzip')
print('Total number of apartments in ' + folder +" is: {}".format(len(file_tmp)))
file_tmp.describe()

Total number of apartments in Chicago-Illinois-US is: 5207


,id,scrape_id,host_id,host_acceptance_rate,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month
count,5.207000e+03,5.207000e+03,5.207000e+03,0.0,5207.000000,5207.000000,0.0,5207.000000,5207.000000,5207.000000,...,5207.000000,4469.000000,4466.000000,4466.000000,4460.000000,4464.000000,4460.000000,4457.000000,5207.000000,4497.000000
mean,1.149098e+07,2.017051e+13,3.596640e+07,NaN,9.987325,9.987325,NaN,41.906928,-87.663527,3.799309,...,25.378721,95.042963,9.703986,9.525526,9.844843,9.863127,9.598655,9.572134,4.065105,2.090536
std,5.320380e+06,1.656409e+00,3.348022e+07,NaN,53.447328,53.447328,NaN,0.054004,0.037147,2.511796,...,38.664955,6.165099,0.655867,0.790706,0.481231,0.428476,0.727752,0.691552,8.218666,1.860132
min,2.384000e+03,2.017051e+13,2.140000e+03,NaN,0.000000,0.000000,NaN,41.657234,-87.844039,1.000000,...,0.000000,20.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,0.010000
25%,7.242074e+06,2.017051e+13,7.689661e+06,NaN,1.000000,1.000000,NaN,41.883807,-87.685719,2.000000,...,2.000000,93.000000,10.000000,9.000000,10.000000,10.000000,9.000000,9.000000,1.000000,0.650000
50%,1.308348e+07,2.017051e+13,2.566896e+07,NaN,1.000000,1.000000,NaN,41.908570,-87.661897,3.000000,...,11.000000,97.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000,1.550000
75%,1.577330e+07,2.017051e+13,5.546114e+07,NaN,3.000000,3.000000,NaN,41.942781,-87.636628,5.000000,...,32.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,3.000000,3.020000
max,1.865680e+07,2.017051e+13,1.296381e+08,NaN,869.000000,869.000000,NaN,42.022298,-87.541280,16.000000,...,410.000000,100.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,56.000000,13.530000


# Open S3 bucket

In [7]:
bucket_name = "chen-gal-test"

In [8]:
s3 = boto3.client("s3")

remote_pathname_l1 = 'AirbnbData/'
remote_pathname_l2 = folder
remote_pathname = remote_pathname_l1 + remote_pathname_l2

# s3.put_object(Body=file , Bucket=bucket_name, Key=remote_pathname)


In [9]:
remote_pathname

'AirbnbData/Chicago-Illinois-US'

In [10]:
logfile_name = 'DataSet/'+'logfile_'+folder+'.txt'
print(f'Logfile name is {logfile_name}')




Logfile name is DataSet/logfile_Chicago-Illinois-US.txt


4870

# Downloading

In [11]:

apt_infos = {}
apt_jses = {}

In [14]:
for i in range(4016,len(file_tmp)):
# for i in range(1):
    print(i)
    url = file_tmp.loc[i,'listing_url']
    apt_id = file_tmp.loc[i,'id']
    apt_name = file_tmp.loc[i,'name']
    with open(logfile_name,'a') as f:
        f.write(f'---------------------\nApartment{i} | {apt_id} | {url} \n')         
    try:
        return_parsed = parse_each_webpage(url)
    except CannotOpenUrl:        
        with open(logfile_name,'a') as f:
            f.writelines('Cannot open '+ str(i)+ ' url.\t' + 'Id: '+ str(apt_id) + ' |  Url: '+ url)                        
        print('Failed to get access to URL, sleep for 20 secs ')
        time.sleep(20)
        return_parsed = parse_each_webpage(url)
    finally:
        scrape_info,ori_json = return_parsed[0],return_parsed[1]
        if len(scrape_info) == 0:
            with open(logfile_name,'a') as f:
                f.writelines('Failed to scrape the url, simply save the json file\n')
        else:
            with open(logfile_name,'a') as f:
                f.writelines('Successfully scraped the url, save both files\n')
            
        dict_apt = {'apt_name':apt_name,'url' : url, 'info': scrape_info}
#         apt_jses.update({str(apt_id):ori_json})
        apt_infos.update({str(apt_id):dict_apt})
        with open(logfile_name,'a') as f:
                f.writelines(f'Finished scraping {i} apartment! ID:{apt_id}\n')
                
#         if i % 75 == 0 and i != 0:            
#             js_filename = remote_pathname +"/jsons/ori_json_"+str(i)            
#             s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=js_filename)
            
#             print("Wrote  down"+str(i)+"json file")
#             with open(logfile_name,'a') as f:
#                 f.writelines(f'Saved json file {js_filename}!\n') 
        if i % 200 == 0 and i != 0:            
            wsdata_filename = remote_pathname +"/ws_data/scraped_partial_data_"+str(i)            
            s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=wsdata_filename)
            
            print("Wrote  down"+str(i)+"ws_data")
            with open(logfile_name,'a') as f:
                f.writelines(f'Saved ws_data file {wsdata_filename}!\n')  
            
        time.sleep(random.random()+2)
        
#upload log_file to s3
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)


4016
request_url:https://www.airbnb.com/rooms/15669026
4017
request_url:https://www.airbnb.com/rooms/1806368
4018
request_url:https://www.airbnb.com/rooms/15798238
4019
request_url:https://www.airbnb.com/rooms/2117415
4020
request_url:https://www.airbnb.com/rooms/2117388
4021
request_url:https://www.airbnb.com/rooms/16831457
4022
request_url:https://www.airbnb.com/rooms/6558625
4023
request_url:https://www.airbnb.com/rooms/16175923
4024
request_url:https://www.airbnb.com/rooms/17666567
4025
request_url:https://www.airbnb.com/rooms/9032209
4026
request_url:https://www.airbnb.com/rooms/7075989
4027
request_url:https://www.airbnb.com/rooms/10768189
4028
request_url:https://www.airbnb.com/rooms/14126256
4029
request_url:https://www.airbnb.com/rooms/16364317
4030
request_url:https://www.airbnb.com/rooms/7692835
4031
request_url:https://www.airbnb.com/rooms/8669608
4032
request_url:https://www.airbnb.com/rooms/9403751
4033
request_url:https://www.airbnb.com/rooms/7395368
4034
request_url:htt

4167
request_url:https://www.airbnb.com/rooms/14855907
4168
request_url:https://www.airbnb.com/rooms/15782924
4169
request_url:https://www.airbnb.com/rooms/8147771
4170
request_url:https://www.airbnb.com/rooms/12243431
4171
request_url:https://www.airbnb.com/rooms/18190669
4172
request_url:https://www.airbnb.com/rooms/11768586
4173
request_url:https://www.airbnb.com/rooms/18566998
4174
request_url:https://www.airbnb.com/rooms/14946095
4175
request_url:https://www.airbnb.com/rooms/10341284
4176
request_url:https://www.airbnb.com/rooms/6768827
4177
request_url:https://www.airbnb.com/rooms/6736478
4178
request_url:https://www.airbnb.com/rooms/4278818
4179
request_url:https://www.airbnb.com/rooms/1984545
4180
request_url:https://www.airbnb.com/rooms/13076417
4181
request_url:https://www.airbnb.com/rooms/13571734
4182
request_url:https://www.airbnb.com/rooms/11430883
4183
request_url:https://www.airbnb.com/rooms/5850897
4184
request_url:https://www.airbnb.com/rooms/15484515
4185
request_url

4317
request_url:https://www.airbnb.com/rooms/11647531
4318
request_url:https://www.airbnb.com/rooms/13127930
4319
request_url:https://www.airbnb.com/rooms/16039516
4320
request_url:https://www.airbnb.com/rooms/32346
4321
request_url:https://www.airbnb.com/rooms/233933
4322
request_url:https://www.airbnb.com/rooms/2310050
4323
request_url:https://www.airbnb.com/rooms/13466713
4324
request_url:https://www.airbnb.com/rooms/18429930
4325
request_url:https://www.airbnb.com/rooms/16823279
4326
request_url:https://www.airbnb.com/rooms/14529530
4327
request_url:https://www.airbnb.com/rooms/18123049
4328
request_url:https://www.airbnb.com/rooms/7074653
4329
request_url:https://www.airbnb.com/rooms/8794937
4330
request_url:https://www.airbnb.com/rooms/11139541
4331
request_url:https://www.airbnb.com/rooms/12579630
4332
request_url:https://www.airbnb.com/rooms/5223619
4333
request_url:https://www.airbnb.com/rooms/14250775
4334
request_url:https://www.airbnb.com/rooms/6419072
4335
request_url:htt

4467
request_url:https://www.airbnb.com/rooms/14578426
4468
request_url:https://www.airbnb.com/rooms/18134292
4469
request_url:https://www.airbnb.com/rooms/15594719
4470
request_url:https://www.airbnb.com/rooms/8188570
4471
request_url:https://www.airbnb.com/rooms/1716626
4472
request_url:https://www.airbnb.com/rooms/15328919
4473
request_url:https://www.airbnb.com/rooms/3616975
4474
request_url:https://www.airbnb.com/rooms/12464708
4475
request_url:https://www.airbnb.com/rooms/13807243
4476
request_url:https://www.airbnb.com/rooms/9679005
4477
request_url:https://www.airbnb.com/rooms/4698247
4478
request_url:https://www.airbnb.com/rooms/15314428
4479
request_url:https://www.airbnb.com/rooms/14446351
4480
request_url:https://www.airbnb.com/rooms/2879976
4481
request_url:https://www.airbnb.com/rooms/12734939
4482
request_url:https://www.airbnb.com/rooms/13915544
4483
request_url:https://www.airbnb.com/rooms/7432064
4484
request_url:https://www.airbnb.com/rooms/10971010
4485
request_url:

4617
request_url:https://www.airbnb.com/rooms/14154000
4618
request_url:https://www.airbnb.com/rooms/7243355
4619
request_url:https://www.airbnb.com/rooms/8118084
4620
request_url:https://www.airbnb.com/rooms/10971626
4621
request_url:https://www.airbnb.com/rooms/12090956
4622
request_url:https://www.airbnb.com/rooms/15925524
4623
request_url:https://www.airbnb.com/rooms/14823659
4624
request_url:https://www.airbnb.com/rooms/10411427
4625
request_url:https://www.airbnb.com/rooms/9690523
4626
request_url:https://www.airbnb.com/rooms/12467112
4627
request_url:https://www.airbnb.com/rooms/10661438
4628
request_url:https://www.airbnb.com/rooms/13332282
4629
request_url:https://www.airbnb.com/rooms/2292298
4630
request_url:https://www.airbnb.com/rooms/14429433
4631
request_url:https://www.airbnb.com/rooms/15925996
4632
request_url:https://www.airbnb.com/rooms/6957805
4633
request_url:https://www.airbnb.com/rooms/9136731
4634
request_url:https://www.airbnb.com/rooms/15926153
4635
request_url

4768
request_url:https://www.airbnb.com/rooms/3898355
4769
request_url:https://www.airbnb.com/rooms/13825159
4770
request_url:https://www.airbnb.com/rooms/15530743
4771
request_url:https://www.airbnb.com/rooms/15530744
4772
request_url:https://www.airbnb.com/rooms/5928976
4773
request_url:https://www.airbnb.com/rooms/16720354
4774
request_url:https://www.airbnb.com/rooms/11040622
4775
request_url:https://www.airbnb.com/rooms/18638982
4776
request_url:https://www.airbnb.com/rooms/5273710
4777
request_url:https://www.airbnb.com/rooms/11187498
4778
request_url:https://www.airbnb.com/rooms/2670087
4779
request_url:https://www.airbnb.com/rooms/8827744
4780
request_url:https://www.airbnb.com/rooms/12039265
4781
request_url:https://www.airbnb.com/rooms/17480697
4782
request_url:https://www.airbnb.com/rooms/3016830
4783
request_url:https://www.airbnb.com/rooms/18134783
4784
request_url:https://www.airbnb.com/rooms/12039993
4785
request_url:https://www.airbnb.com/rooms/14359176
4786
request_url

4918
request_url:https://www.airbnb.com/rooms/8747106
4919
request_url:https://www.airbnb.com/rooms/4503103
4920
request_url:https://www.airbnb.com/rooms/15379941
4921
request_url:https://www.airbnb.com/rooms/18161127
4922
request_url:https://www.airbnb.com/rooms/4503332
4923
request_url:https://www.airbnb.com/rooms/18341416
4924
request_url:https://www.airbnb.com/rooms/11661652
4925
request_url:https://www.airbnb.com/rooms/15499376
4926
request_url:https://www.airbnb.com/rooms/17881850
4927
request_url:https://www.airbnb.com/rooms/10973773
4928
request_url:https://www.airbnb.com/rooms/14692242
4929
request_url:https://www.airbnb.com/rooms/8580517
4930
request_url:https://www.airbnb.com/rooms/16440984
4931
request_url:https://www.airbnb.com/rooms/7783924
4932
request_url:https://www.airbnb.com/rooms/16620615
4933
request_url:https://www.airbnb.com/rooms/13362992
4934
request_url:https://www.airbnb.com/rooms/17111165
4935
request_url:https://www.airbnb.com/rooms/16396651
4936
request_ur

5068
request_url:https://www.airbnb.com/rooms/773127
5069
request_url:https://www.airbnb.com/rooms/4457161
5070
request_url:https://www.airbnb.com/rooms/934335
5071
request_url:https://www.airbnb.com/rooms/15548581
5072
request_url:https://www.airbnb.com/rooms/985292
5073
request_url:https://www.airbnb.com/rooms/18344621
5074
request_url:https://www.airbnb.com/rooms/207351
5075
request_url:https://www.airbnb.com/rooms/207218
5076
request_url:https://www.airbnb.com/rooms/4925062
5077
request_url:https://www.airbnb.com/rooms/1843765
5078
request_url:https://www.airbnb.com/rooms/18091194
5079
request_url:https://www.airbnb.com/rooms/15009956
5080
request_url:https://www.airbnb.com/rooms/8985325
5081
request_url:https://www.airbnb.com/rooms/5996942
5082
request_url:https://www.airbnb.com/rooms/10598914
5083
request_url:https://www.airbnb.com/rooms/13311396
5084
request_url:https://www.airbnb.com/rooms/11475540
5085
request_url:https://www.airbnb.com/rooms/16618981
5086
request_url:https://

In [15]:
logfile_s3_filename = remote_pathname+'/ws_data/log_file.txt'
s3.upload_file(Bucket = bucket_name,Filename=logfile_name,Key=logfile_s3_filename)

In [102]:
with open(logfile_name,'a') as f:
    f.writelines(f'Saved json file {js_filename}!\n')                            
jses = {}

## Save the result

### save scrapted data

In [16]:
scraped_filename = remote_pathname +"/ws_data/webscrapted.json"

s3.put_object(Body=json.dumps(apt_infos) , Bucket=bucket_name, Key=scraped_filename)

{'ETag': '"e5be8054bac84d9f037ecc369719eb7d"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Wed, 02 May 2018 04:27:30 GMT',
   'etag': '"e5be8054bac84d9f037ecc369719eb7d"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'rjbHFiw1U3qRRNSXOpal38U9mq2s7b7uRxZNKlUeZvrbBGwWPBGXSUeC4HBQ13A30POoEh8E8c8=',
   'x-amz-request-id': 'FA72BE06AE13EEC8'},
  'HTTPStatusCode': 200,
  'HostId': 'rjbHFiw1U3qRRNSXOpal38U9mq2s7b7uRxZNKlUeZvrbBGwWPBGXSUeC4HBQ13A30POoEh8E8c8=',
  'RequestId': 'FA72BE06AE13EEC8',
  'RetryAttempts': 0}}

### save json data

In [88]:
# last_piece_js_filename = remote_pathname+"/jsons/ori_json_lastpiece.json"

# s3.put_object(Body=json.dumps(apt_jses) , Bucket=bucket_name, Key=last_piece_js_filename)


{'ETag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
   'date': 'Mon, 30 Apr 2018 01:22:39 GMT',
   'etag': '"7b336feb550bfad1dc2c55de6ec8a93d"',
   'server': 'AmazonS3',
   'x-amz-id-2': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
   'x-amz-request-id': 'BB474E35A45B7DAC'},
  'HTTPStatusCode': 200,
  'HostId': 'S7BG7n3tOOJQXdwejYwoPOwpqzfEF9NGChTaAzo/bBZ5ZmmQJvaioLk2y3mYnDQtAsnUZ52XF/k=',
  'RequestId': 'BB474E35A45B7DAC',
  'RetryAttempts': 0}}

## Load json data from s3

In [11]:
scraped_filename = 'AirbnbData/Boston-Massachusetts-US/ws_data/scraped_partial_data_3600'

response = s3.get_object(Bucket=bucket_name,
                         Key=scraped_filename)

In [19]:
response

{'AcceptRanges': 'bytes',
 'Body': <botocore.response.StreamingBody at 0x7fee21cc0d30>,
 'ContentLength': 5059688,
 'ContentType': 'binary/octet-stream',
 'ETag': '"3aa75aea13d6b1d083cea56b1332defc"',
 'LastModified': datetime.datetime(2018, 4, 30, 14, 25, 10, tzinfo=tzutc()),
 'Metadata': {},
 'ResponseMetadata': {'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-length': '5059688',
   'content-type': 'binary/octet-stream',
   'date': 'Tue, 01 May 2018 05:46:32 GMT',
   'etag': '"3aa75aea13d6b1d083cea56b1332defc"',
   'last-modified': 'Mon, 30 Apr 2018 14:25:10 GMT',
   'server': 'AmazonS3',
   'x-amz-id-2': 'VgQXishVBTFtclbB8DJ3hcIxNKV9hLHaJQwz+0jizaRHyz3WHejmgO/NCmR2NeXNjU4CMeGdTJo=',
   'x-amz-request-id': 'D35DCF8A8B8701D1'},
  'HTTPStatusCode': 200,
  'HostId': 'VgQXishVBTFtclbB8DJ3hcIxNKV9hLHaJQwz+0jizaRHyz3WHejmgO/NCmR2NeXNjU4CMeGdTJo=',
  'RequestId': 'D35DCF8A8B8701D1',
  'RetryAttempts': 0}}

In [12]:
scraped_data = json.loads(response['Body'].read())

In [13]:
apt_infos = scraped_data

In [14]:
len(apt_infos)

3600